In [1]:
#test on rl worl_model net
from metaworld.envs import (ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE)
from bnn import BNN
import numpy as np
import torch.nn as nn
import torch

In [2]:
DEVICE = 'cuda:0'

In [3]:
torch.set_default_dtype(torch.float64)
class_env = "door-open-v2-goal-observable"
door_open_goal_observable_cls = ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE[class_env]
env = door_open_goal_observable_cls()

action_space_shape = env.action_space.shape[0]
obs_space_shape = env.observation_space.shape[0]

/home/dema/PycharmProjects/lifelong_rl/venv/lib/python3.10/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float64
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/dema/PycharmProjects/lifelong_rl/venv/lib/python3.10/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [4]:
s = env.reset()
dyna = BNN(action_space_shape,
            obs_space_shape,
            reward_dim=1).to(DEVICE)
world_model_state = torch.load('model_stock/world/model_envWorld_small_kl_checkpoint80000.pth')
dyna.copy_params_from_world_model(world_model_state)

In [5]:
act = torch.randn(4)
X = torch.cat([torch.from_numpy(s), act]).to(DEVICE)
dyna(X)

tensor([-0.1304,  0.9361, -0.1644,  0.3671, -1.1246,  0.2806,  0.2292,  0.3678,
         0.8632,  0.5102,  1.5793, -0.5993, -0.3602,  0.7561, -0.2607, -0.3539,
        -0.2226,  0.0397,  0.8898,  0.7100, -0.1245, -0.6392, -1.0275,  0.1725,
        -0.8990,  0.5207,  0.1441,  0.6406, -0.2344, -0.6517, -0.4486, -0.2282,
         0.3794,  0.2772,  0.2232,  0.0810,  0.4103,  0.5063,  0.0221, -0.5671],
       device='cuda:0', grad_fn=<AddBackward0>)

In [8]:
from multi_lev_nn import Propagation_net
from cem_optimizer_v2 import CEM_opt

num_particle = 50
cem = CEM_opt(num_particle)
prop_net = Propagation_net(num_particle)
prop_net.model_to_gpu()



In [9]:
import time

In [10]:
s = env.reset()
rewards_for_ep = np.zeros((1, 2))
t = time.time()
# 1 full planning step:
for ep in range(1):
    
    horizon_rewards = np.zeros(2)
    for h in range(2): # plan horizon
        s = torch.from_numpy(s).to(DEVICE)

        #plan step
        act_sequences = torch.from_numpy(cem.population).to(DEVICE)
        plan_rewards = np.zeros(act_sequences.shape[0])   
        for idx, act_seq in enumerate(act_sequences):
            act_seq = act_seq.to(DEVICE)
            prop_net.sample_from(dyna)
            plan_rewards[idx] = prop_net.propagate(s, act_seq ).cpu().detach().numpy()
        cem.update(plan_rewards)

        best_act = cem.solutions()[:action_space_shape]
        s, r, d, info = env.step(best_act)
        horizon_rewards[h] = r
        
    rewards_for_ep[ep] = horizon_rewards

print('time for 10 ep:', time.time() - t)


time for 10 ep: 6.701117038726807


In [11]:
plan_rewards

array([0.38449622, 0.42597993, 0.40276965, 0.27138755, 0.41832809,
       0.6423669 , 0.41482357, 0.6845484 , 0.34421033, 0.34833691,
       0.467046  , 0.55262818, 0.469669  , 0.29760448, 0.44785983,
       0.36239813, 0.43423505, 0.36475662, 0.43022591, 0.31058636,
       0.23653314, 0.46944177, 0.32913878, 0.3919809 , 0.60805637,
       0.3415954 , 0.29135307, 0.47940163, 0.31924628, 0.49533109,
       0.37401359, 0.25312306, 0.39187549, 0.36872228, 0.28845616,
       0.23088496, 0.52820339, 0.86821384, 0.34632887, 0.56968696,
       0.31219066, 0.44370236, 0.44469619, 0.45990277, 0.33693032,
       0.56292083, 0.3197465 , 0.55353843, 0.49008214, 0.41539506])

0.6634128223974651